In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/birdclef-2024/sample_submission.csv
/kaggle/input/birdclef-2024/eBird_Taxonomy_v2021.csv
/kaggle/input/birdclef-2024/train_metadata.csv
/kaggle/input/birdclef-2024/unlabeled_soundscapes/1872382287.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/161691311.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/884633066.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/1675425185.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/1469579221.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/1564606141.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/194233408.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/2003586554.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/1459749597.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/33854460.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/1163678028.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/1682474551.ogg
/kaggle/input/birdclef-2024/unlabeled_soundscapes/825970922.ogg
/kaggle/inp

In [2]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # "jax" or "tensorflow" or "torch" 

import keras_cv
import keras
import keras.backend as K
import tensorflow as tf
import tensorflow_io as tfio

import numpy as np 
import pandas as pd

from glob import glob
from tqdm import tqdm

import librosa
import IPython.display as ipd
import librosa.display as lid

import matplotlib.pyplot as plt
import matplotlib as mpl

cmap = mpl.cm.get_cmap('coolwarm')

2024-05-29 04:57:41.903768: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-29 04:57:41.903886: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-29 04:57:42.039799: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_18/1585041062.py:23: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = mpl.cm.get_cmap('coolwarm')


In [3]:
class CFG:
    seed = 42
    
    # Input image size and batch size
    img_size = [128, 384]
    
    # Audio duration, sample rate, and length
    duration = 15 # second
    sample_rate = 32000
    audio_len = duration*sample_rate
    
    # STFT parameters
    nfft = 2028
    window = 2048
    hop_length = audio_len // (img_size[1] - 1)
    fmin = 20
    fmax = 16000
    
    # Number of epochs, model name
    preset = 'efficientnetv2_b2_imagenet'

    # Class Labels for BirdCLEF 24
    class_names = sorted(os.listdir('/kaggle/input/birdclef-2024/train_audio/'))
    num_classes = len(class_names)
    class_labels = list(range(num_classes))
    label2name = dict(zip(class_labels, class_names))
    name2label = {v:k for k,v in label2name.items()}

In [4]:
tf.keras.utils.set_random_seed(CFG.seed)

In [5]:
BASE_PATH = '/kaggle/input/birdclef-2024'

In [6]:
test_paths = glob(f'{BASE_PATH}/test_soundscapes/*ogg')
# During commit use `unlabeled` data as there is no `test` data.
# During submission `test` data will automatically be populated.
if len(test_paths)==0:
    test_paths = glob(f'{BASE_PATH}/unlabeled_soundscapes/*ogg')[:10]
test_df = pd.DataFrame(test_paths, columns=['filepath'])
test_df.head()

,filepath
0,/kaggle/input/birdclef-2024/unlabeled_soundsca...
1,/kaggle/input/birdclef-2024/unlabeled_soundsca...
2,/kaggle/input/birdclef-2024/unlabeled_soundsca...
3,/kaggle/input/birdclef-2024/unlabeled_soundsca...
4,/kaggle/input/birdclef-2024/unlabeled_soundsca...


In [7]:
# Create an input layer for the model
inp = keras.layers.Input(shape=(None, None, 3))
# Pretrained backbone
backbone = keras_cv.models.EfficientNetV2Backbone.from_preset(
    CFG.preset,
)
out = keras_cv.models.ImageClassifier(
    backbone=backbone,
    num_classes=CFG.num_classes,
    name="classifier"
)(inp)
# Build model
model = keras.models.Model(inputs=inp, outputs=out)
# Load weights of trained model
model.load_weights("/kaggle/input/baseline-weights/best_model.weights.h5")

Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/efficientnetv2/keras/efficientnetv2_b2_imagenet/2' to your Kaggle notebook...


In [8]:
# Decodes Audio
def build_decoder(with_labels=True, dim=1024):
    def get_audio(filepath):
        file_bytes = tf.io.read_file(filepath)
        audio = tfio.audio.decode_vorbis(file_bytes) # decode .ogg file
        audio = tf.cast(audio, tf.float32)
        if tf.shape(audio)[1]>1: # stereo -> mono
            audio = audio[...,0:1]
        audio = tf.squeeze(audio, axis=-1)
        return audio
    
    def create_frames(audio, duration=5, sr=32000):
        frame_size = int(duration * sr)
        audio = tf.pad(audio[..., None], [[0, tf.shape(audio)[0] % frame_size], [0, 0]]) # pad the end
        audio = tf.squeeze(audio) # remove extra dimension added for padding
        frames = tf.reshape(audio, [-1, frame_size]) # shape: [num_frames, frame_size]
        return frames
    
    def apply_preproc(spec):
        # Standardize
        mean = tf.math.reduce_mean(spec)
        std = tf.math.reduce_std(spec)
        spec = tf.where(tf.math.equal(std, 0), spec - mean, (spec - mean) / std)

        # Normalize using Min-Max
        min_val = tf.math.reduce_min(spec)
        max_val = tf.math.reduce_max(spec)
        spec = tf.where(tf.math.equal(max_val - min_val, 0), spec - min_val,
                              (spec - min_val) / (max_val - min_val))
        return spec

    def decode(path):
        # Load audio file
        audio = get_audio(path)
        # Split audio file into frames with each having 5 seecond duration
        audio = create_frames(audio)
        # Convert audio to spectrogram
        spec = keras.layers.MelSpectrogram(num_mel_bins=CFG.img_size[0],
                                             fft_length=CFG.nfft, 
                                              sequence_stride=CFG.hop_length, 
                                              sampling_rate=CFG.sample_rate)(audio)
        # Apply normalization and standardization
        spec = apply_preproc(spec)
        # Covnert spectrogram to 3 channel image (for imagenet)
        spec = tf.tile(spec[..., None], [1, 1, 1, 3])
        return spec
    
    return decode

In [9]:
def build_dataset(paths, batch_size=1, decode_fn=None, cache=False):
    if decode_fn is None:
        decode_fn = build_decoder(dim=CFG.audio_len) # decoder
    AUTO = tf.data.experimental.AUTOTUNE
    slices = (paths,)
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.map(decode_fn, num_parallel_calls=AUTO) # decode audio to spectrograms then create frames
    ds = ds.cache() if cache else ds # cache files
    ds = ds.batch(batch_size, drop_remainder=False) # create batches
    ds = ds.prefetch(AUTO)
    return ds

In [10]:
# Initialize empty list to store ids
ids = []

# Initialize empty array to store predictions
preds = np.empty(shape=(0, CFG.num_classes), dtype='float32')

# Build test dataset
test_paths = test_df.filepath.tolist()
test_ds = build_dataset(paths=test_paths, batch_size=1)

# Iterate over each audio file in the test dataset
for idx, specs in enumerate(tqdm(iter(test_ds), desc='test ', total=len(test_df))):
    # Extract the filename without the extension
    filename = test_paths[idx].split('/')[-1].replace('.ogg','')
    
    # Convert to backend-specific tensor while excluding extra dimension
    specs = keras.ops.convert_to_tensor(specs[0])
    
    # Predict bird species for all frames in a recording using all trained models
    frame_preds = model.predict(specs, verbose=0)
    
    # Create a ID for each frame in a recording using the filename and frame number
    frame_ids = [f'{filename}_{(frame_id+1)*5}' for frame_id in range(len(frame_preds))]
    
    # Concatenate the ids
    ids += frame_ids
    # Concatenate the predictions
    preds = np.concatenate([preds, frame_preds], axis=0)

test : 100%|██████████| 10/10 [00:21<00:00,  2.16s/it]


In [11]:
# Submit prediction
pred_df = pd.DataFrame(ids, columns=['row_id'])
pred_df.loc[:, CFG.class_names] = preds
pred_df.to_csv('submission.csv',index=False)
pred_df.head()

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1
0,1872382287_5,0.006214,0.010947,0.009938,0.000350,0.003966,0.000192,0.001516,0.000861,0.003002,...,0.003546,0.000852,0.002408,0.001774,0.009406,0.000732,0.000499,0.000242,0.000370,0.000570
1,1872382287_10,0.004327,0.005579,0.001050,0.000599,0.008579,0.000995,0.013172,0.000899,0.009792,...,0.003166,0.002472,0.008012,0.004146,0.004372,0.003542,0.005511,0.000861,0.001877,0.002110
2,1872382287_15,0.000522,0.006903,0.000181,0.000432,0.001286,0.000400,0.029445,0.000315,0.009412,...,0.002503,0.002705,0.002123,0.004192,0.000595,0.000387,0.008752,0.000943,0.007745,0.000208
3,1872382287_20,0.004520,0.001876,0.001937,0.000223,0.014672,0.000535,0.001320,0.000635,0.006552,...,0.000995,0.001510,0.002705,0.003944,0.001247,0.006303,0.000698,0.002012,0.002399,0.004076
4,1872382287_25,0.003628,0.002565,0.000773,0.000625,0.016268,0.000874,0.005461,0.000576,0.003891,...,0.002309,0.003572,0.006107,0.003623,0.001337,0.004432,0.001542,0.000862,0.003249,0.001539
